# ANALYZING RTC SEVERITY DATASET
In this project, we are analyzing rioad accident data in order to answer the following questions:
1. When do most road accidents occur? What hour?
2. Which roads lead to major road accidents.
3. Which vehicles lead to fatal or serious accidents?

In [1]:
import csv
import datetime as dt

In [2]:
# read the csv file
with open("C://Users//user//Downloads//accident_data.csv") as file:
    rows=list(csv.reader(file))

In [3]:
print(rows[:5])

[['Accident_Index', '1st_Road_Class', '1st_Road_Number', '2nd_Road_Class', '2nd_Road_Number', 'Accident_Severity', 'Carriageway_Hazards', 'Date', 'Day_of_Week', 'Did_Police_Officer_Attend_Scene_of_Accident', 'Junction_Control', 'Junction_Detail', 'Latitude', 'Light_Conditions', 'Local_Authority_(District)', 'Local_Authority_(Highway)', 'Location_Easting_OSGR', 'Location_Northing_OSGR', 'Longitude', 'LSOA_of_Accident_Location', 'Number_of_Casualties', 'Number_of_Vehicles', 'Pedestrian_Crossing-Human_Control', 'Pedestrian_Crossing-Physical_Facilities', 'Police_Force', 'Road_Surface_Conditions', 'Road_Type', 'Special_Conditions_at_Site', 'Speed_limit', 'Time', 'Urban_or_Rural_Area', 'Weather_Conditions', 'Year', 'InScotland'], ['200501BS00001', 'A', '3218', '', '0', 'Serious', 'None', '04/01/2005', 'Tuesday', '1', 'Data missing or out of range', 'Not at junction or within 20 metres', '51.489096', '', 'Kensington and Chelsea', 'Kensington and Chelsea', '525680', '178240', '-0.19117', 'E010

In [4]:
header = rows[0]
print(len(header))

34


In [5]:
data = rows[1:]
print(len(data[0]))

34


In [6]:
# function to explore the dataset to find out how many rows and columns there are
def explore_dataset(dataset, start, end, rows_columns = False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print("\n")
    if rows_columns:
        print("Number of rows",len(dataset))
        print("Number of columns", len(dataset[0]))

In [7]:
explore_dataset(data, 0,3,True)

['200501BS00001', 'A', '3218', '', '0', 'Serious', 'None', '04/01/2005', 'Tuesday', '1', 'Data missing or out of range', 'Not at junction or within 20 metres', '51.489096', '', 'Kensington and Chelsea', 'Kensington and Chelsea', '525680', '178240', '-0.19117', 'E01002849', '1', '1', '0', '1', 'Metropolitan Police', 'Wet or damp', 'Single carriageway', 'None', '30', '17:42', 'Urban', 'Raining no high winds', '2005', 'No']


['200501BS00002', 'B', '450', 'C', '0', 'Slight', 'None', '05/01/2005', 'Wednesday', '1', 'Auto traffic signal', 'Crossroads', '51.520075', 'Darkness - lights lit', 'Kensington and Chelsea', 'Kensington and Chelsea', '524170', '181650', '-0.211708', 'E01002909', '1', '1', '0', '5', 'Metropolitan Police', 'Dry', 'Dual carriageway', 'None', '30', '17:36', 'Urban', 'Fine no high winds', '2005', 'No']


['200501BS00003', 'C', '0', '', '0', 'Slight', 'None', '06/01/2005', 'Thursday', '1', 'Data missing or out of range', 'Not at junction or within 20 metres', '51.525301', 

In [8]:
# check for wrong data in my lists of lists
def check_wrong_data(mylist):
    for row in mylist:
        header_length = len(header)
        row_length = len(row)
        if row_length != header_length:
            print(row)
            print(data.index(row)) 

In [9]:
check_wrong_data(data)

In [10]:
roadtypes = []
for row in data:
    road_type = row[1]
    if road_type not in roadtypes:
        roadtypes.append(road_type)
print(roadtypes)

['A', 'B', 'C', '', 'Motorway', 'A(M)']


In [11]:
# check for duplicates
duplicate_entries = []
unique_entries = []
for row in data:
    accident_id = row[0]
    if accident_id in unique_entries:
        date_day_dicticate_entries.append(accident_id)
#     else:
#         unique_entries.append(accident_id)
# len_unique_entries = len(unique_entries)
len_duplicate_entries = len(duplicate_entries)
# example_duplicate = duplicate_entries[3:5]
# print("There are", len_unique_entries, "unique entries")
print("There are", len_duplicate_entries, "duplicate entries")
# print(example_duplicate)
#     return len_duplicate_entries, len_unique_entries, example_duplicate
# len_duplicate_entries, len_unique_entries, example_duplicate = check_duplicates(data)
# print(
#     f"Number of duplicate entries {len_duplicate_entries}\n"
#     f"Number of unique entries  {len_unique_entries}\n"
#     f"Examples of duplicate entries {example_duplicate}\n"
#      )

There are 0 duplicate entries


In [12]:
# fill in the empty strings with the string(unknown Data)
def fill_missing_strings(i):
    for row in data:
        col = row[i]
        col = col.title()
        if not col:
            col = "Unknown Data"
        row[i] = col

In [13]:
for i in range(len(header)):
    fill_missing_strings(i)
print(data[2:5])

[['200501Bs00003', 'C', '0', 'Unknown Data', '0', 'Slight', 'None', '06/01/2005', 'Thursday', '1', 'Data Missing Or Out Of Range', 'Not At Junction Or Within 20 Metres', '51.525301', 'Darkness - Lights Lit', 'Kensington And Chelsea', 'Kensington And Chelsea', '524520', '182240', '-0.206458', 'E01002857', '1', '2', '0', '0', 'Metropolitan Police', 'Dry', 'Single Carriageway', 'None', '30', '00:15', 'Urban', 'Fine No High Winds', '2005', 'No'], ['200501Bs00004', 'A', '3220', 'Unknown Data', '0', 'Slight', 'None', '07/01/2005', 'Friday', '1', 'Data Missing Or Out Of Range', 'Not At Junction Or Within 20 Metres', '51.482442', 'Unknown Data', 'Kensington And Chelsea', 'Kensington And Chelsea', '526900', '177530', '-0.173862', 'E01002840', '1', '1', '0', '0', 'Metropolitan Police', 'Dry', 'Single Carriageway', 'None', '30', '10:35', 'Urban', 'Fine No High Winds', '2005', 'No'], ['200501Bs00005', 'Unknown Data', '0', 'Unknown Data', '0', 'Slight', 'None', '10/01/2005', 'Monday', '1', 'Data Mi

In [14]:
# get the index of date column
print("The index of Date column is ",header.index("Date"))
print("The index of Time column is ",header.index("Time"))
print("The index of Year column is ",header.index("Year"))
print("The index of no. of casualties column is ",header.index("Number_of_Casualties"))


The index of Date column is  7
The index of Time column is  29
The index of Year column is  32
The index of no. of casualties column is  20


In [15]:
for row in data:
    myyear = row[32]
    mydates = row[7]
print(type(myyear))

<class 'str'>


In [16]:
# parse strings as dates
for row in data:
    mydates = row[7]
    mydates = dt.datetime.strptime(mydates, "%d/%m/%Y").date()
    row[7] = mydates

    mytime = row[29]
#     replace Unknown Data Time with 00:00
    mytime=mytime.replace("Unknown Data","00:00")
    mytime = dt.datetime.strptime(mytime, "%H:%M").time()
    row[29] = mytime
    
    myyear = row[32]
    myyear = dt.datetime.strptime(myyear, "%Y").year
    row[32] = myyear
    
print(type(row[7]))
print(type(row[29]))
print(type(row[32]))

<class 'datetime.date'>
<class 'datetime.time'>
<class 'int'>


In [17]:
dates_1 = []
day_1 = []
for row in data:    
    mydate = row[7]
    dates_1.append(mydate)
    day_of_week = row[8]
    day_1.append(day_of_week)
print(len(dates_1))
print(len(day_1))

# Using the `zip` function to combine the two lists into a list of tuples
date_data = list(zip(dates_1, day_1))

# Converting the list of tuples to a dictionary
date_dict = dict(date_data)
# print the first few elements of the dictionary 
first_five = dict(list(date_dict.items())[:4])

print(first_five)

# Converting the dictionary to a list of dictionaries
dict_list = [{'Date': key, 'Day Of Week': value} for key, value in date_dict.items()]
print(dict_list[2:5])

1048575
1048575
{datetime.date(2005, 1, 4): 'Tuesday', datetime.date(2005, 1, 5): 'Wednesday', datetime.date(2005, 1, 6): 'Thursday', datetime.date(2005, 1, 7): 'Friday'}
[{'Date': datetime.date(2005, 1, 6), 'Day Of Week': 'Thursday'}, {'Date': datetime.date(2005, 1, 7), 'Day Of Week': 'Friday'}, {'Date': datetime.date(2005, 1, 10), 'Day Of Week': 'Monday'}]


In [18]:
print(data[100:104])

[['200501Bs70083', 'B', '450', 'Unknown Data', '0', 'Slight', 'None', datetime.date(2005, 3, 17), 'Thursday', '1', 'Data Missing Or Out Of Range', 'Not At Junction Or Within 20 Metres', '51.517263', 'Unknown Data', 'Kensington And Chelsea', 'Kensington And Chelsea', '524290', '181340', '-0.210089', 'E01002831', '1', '1', '0', '4', 'Metropolitan Police', 'Dry', 'Single Carriageway', 'None', '30', datetime.time(9, 15), 'Urban', 'Fine No High Winds', 2005, 'No'], ['200501Bs70084', 'B', '450', 'Unclassified', '0', 'Slight', 'None', datetime.date(2005, 3, 13), 'Sunday', '3', 'Give Way Or Uncontrolled', 'Crossroads', '51.516353', 'Darkness - Lights Lit', 'Kensington And Chelsea', 'Kensington And Chelsea', '524340', '181240', '-0.209404', 'E01002879', '1', '2', '0', '0', 'Metropolitan Police', 'Dry', 'Single Carriageway', 'None', '30', datetime.time(0, 15), 'Urban', 'Fine No High Winds', 2005, 'No'], ['200501Bs70085', 'C', '0', 'Unknown Data', '0', 'Slight', 'None', datetime.date(2005, 3, 15)

In [24]:
# which day had the most accidents?
result_list = []
for row in data:
    accident_at = row[7]
    no_casualties = int(row[20])
    result_list.append([accident_at,no_casualties])
accidents_by_day = {}
casualties_by_day = {}

for result in result_list:
    mydates = result[0]
    casualties = result[1]
    if mydates in accidents_by_day:
        accidents_by_day[mydates] += 1
        casualties_by_day[mydates] += casualties
    else:
        accidents_by_day[mydates] = 1
        casualties_by_day[mydates] = casualties
# first 5 key:value pairs 
import more_itertools
first_few = more_itertools.take(5, casualties_by_day.items())
first_few
# casualties_by_day

[(datetime.date(2005, 1, 4), 599),
 (datetime.date(2005, 1, 5), 686),
 (datetime.date(2005, 1, 6), 710),
 (datetime.date(2005, 1, 7), 656),
 (datetime.date(2005, 1, 10), 688)]

In [26]:
# sort to see the when most accidents occured and the least
Max_casualties = None
Min_casualties = None
for key in casualties_by_day:
    if Max_casualties is None or casualties_by_day[key] > Max_casualties:
        Max_casualties = casualties_by_day[key]
    if Min_casualties is None or casualties_by_day[key] < Max_casualties:
        Min_casualties = casualties_by_day[key]
most_date = None
least_date = None
for key in casualties_by_day:
    if casualties_by_day[key] == Max_casualties:
        most_date = key
    if casualties_by_day[key] == Min_casualties:
        least_date = key
        
print("The most accidents is {} and occured on {}".format(Max_casualties,most_date))
print("The least accidents is {} and occured on {}".format(Min_casualties,least_date))   

The most accidents is 1157 and occured on 2005-10-21
The least accidents is 235 and occured on 2010-12-25
